In [1]:
!pip install -q langchain langchain-community faiss-cpu transformers accelerate sqlalchemy pandas sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


**Create SQLite Supply-Chain DataBase**

In [2]:
import sqlite3

conn = sqlite3.connect("supply_chain.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    quantity INTEGER,
    order_date TEXT,
    delivery_date TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS shipments (
    order_id INTEGER,
    delay_days INTEGER
)
""")

conn.commit()
conn.close()

print("Database created successfully")


Database created successfully


**Prepare Schema Documents (For RAG)**

In [3]:
schema_docs = [
    "Table products(product_id, product_name, category)",
    "Table orders(order_id, product_id, quantity, order_date, delivery_date)",
    "Table shipments(order_id, delay_days)"
]

schema_docs


['Table products(product_id, product_name, category)',
 'Table orders(order_id, product_id, quantity, order_date, delivery_date)',
 'Table shipments(order_id, delay_days)']

**Build Vector STore RAG**

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_texts(schema_docs, embeddings)
retriever = vectorstore.as_retriever()

print("RAG vector store ready")

/tmp/ipython-input-4274095473.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RAG vector store ready


**Load Long-Context Open Source LLM**

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

print("LLM loaded successfully")


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

LLM loaded successfully


**Text To SQL Prompt(Schema-Aware)**

In [6]:
def generate_sql(question):
    docs = retriever.invoke(question)
    schema_context = "\n".join([d.page_content for d in docs])

    prompt = f"""
You are an expert SQL developer.
Use ONLY the schema below.

Schema:
{schema_context}

Rules:
- Generate ONLY SELECT queries
- Do NOT modify data

Question:
{question}

SQL:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the SQL query from the full response
    # The SQL query is expected to be within a markdown code block like ```sql ... ```
    sql_start = full_response.find("```sql")
    if sql_start != -1:
        sql_end = full_response.find("```", sql_start + 6) # +6 to skip ```sql\n
        if sql_end != -1:
            sql = full_response[sql_start + 6:sql_end].strip()
        else:
            # If no closing ``` is found, take everything from the start of ```sql
            sql = full_response[sql_start + 6:].strip()
    else:
        # If no ```sql block is found, assume the entire response is the SQL or an error
        sql = full_response.strip()

    return sql

**SQL Safety Validation**

In [7]:
def is_safe_sql(sql):
    forbidden = ["DROP", "DELETE", "UPDATE", "INSERT", "ALTER"]
    return not any(word in sql.upper() for word in forbidden)

**EXecute SQL (Tool Calling)**

In [8]:
import pandas as pd

def execute_sql(sql):
    if not is_safe_sql(sql):
        return " Unsafe SQL detected"

    conn = sqlite3.connect("supply_chain.db")
    try:
        df = pd.read_sql(sql, conn)
    except Exception as e:
        df = str(e)
    conn.close()

    return df


**Run the TEXT_to_SQL- RAG ChatBOt**

In [13]:
while True:
    question = input("Ask a supply-chain question (type exit to stop): ")
    if question.lower() == "exit":
        break

    sql_query = generate_sql(question)
    print("\nGenerated SQL:\n", sql_query)

    result = execute_sql(sql_query)
    print("\nResult:\n", result)

Ask a supply-chain question (type exit to stop): exit
